In [ ]:
from src.agent import HelpDeskAgent
from src.configs import Settings
from src.tools.search_xyz_manual import search_xyz_manual
from src.tools.search_xyz_qa import (
    search_xyz_qa,
)

settings = Settings()


In [3]:
agent = HelpDeskAgent(
    settings=settings,
    tools=[search_xyz_manual, search_xyz_qa],
)

In [25]:
question = """
Hello,

I am currently considering using the XYZ system and would appreciate your guidance on the following two points.

1. Restrictions on characters allowed for passwords
When setting a password in this system, could you please provide detailed information about the allowed character set (e.g., alphanumeric characters, symbols, length limits, etc.)? In order to ensure security and avoid authentication errors in the system, I would like to confirm the exact specifications.

2. How to obtain the latest release
I would also appreciate it if you could let me know how to check and obtain the latest update/release information.

Thank you very much for your time and support.
"""

# question = """
# Hello,

# I am currently considering using the XYZ system and would appreciate your guidance on the following points.

# 1. How to restrict notifications to only a specific project

# 2. Restrictions on characters allowed for passwords
# When setting a password in this system, could you please provide detailed information about the allowed character set (e.g., alphanumeric characters, symbols, length limits, etc.)? In order to ensure security and avoid authentication errors in the system, I would like to confirm the exact specifications.

# Thank you very much for your time and support.

# """

## Planning Step

In [26]:
input_data = {"question": question}

plan_result = agent.create_plan(state=input_data)

In [ ]:
plan_result["plan"]

## Tool Selection Step

In [28]:
input_data = {
    "question": question,
    "plan": plan_result["plan"],
    "subtask": plan_result["plan"][0],
    "challenge_count": 0,
    "is_completed": False,
}

In [29]:
select_tool_result = agent.select_tools(state=input_data)

In [ ]:
select_tool_result

In [31]:
select_tool_result["messages"][-1]

{'role': 'assistant',
 'tool_calls': [{'id': 'call_zM2g5tqao9s01G4uYwCtplrr',
   'function': {'arguments': '{"keywords":"パスワード 設定 文字制限"}',
    'name': 'search_xyz_manual'},
   'type': 'function'}]}

In [ ]:
select_tool_result["messages"]

## Tool Execution Step

In [33]:
input_data = {
    "question": question,
    "plan": plan_result["plan"],
    "subtask": plan_result["plan"][0],
    "challenge_count": 0,
    "messages": select_tool_result["messages"],
    "is_completed": False,
}

In [ ]:
tool_results = agent.execute_tools(state=input_data)

In [ ]:
tool_results["tool_results"][0][0].results

In [ ]:
tool_results

## Subtask Answer

In [37]:
input_data = {
    "question": question,
    "plan": plan_result["plan"],
    "subtask": plan_result["plan"][0],
    "challenge_count": 0,
    "messages": tool_results["messages"],
    "tool_results": tool_results["tool_results"],
    "is_completed": False,
}

In [38]:
subtask_answer = agent.create_subtask_answer(state=input_data)

In [ ]:
subtask_answer

In [ ]:
print(subtask_answer["subtask_answer"])

## Reflection

In [41]:
input_data = {
    "question": question,
    "plan": plan_result["plan"],
    "subtask": plan_result["plan"][0],
    "challenge_count": 0,
    "messages": subtask_answer["messages"],
    "tool_results": tool_results["tool_results"],
    "is_completed": False,
    "subtask_answer": subtask_answer["subtask_answer"],
}

In [42]:
reflection_result = agent.reflect_subtask(state=input_data)

In [ ]:
reflection_result

In [ ]:
# Check which tool was selected first
print(reflection_result["messages"][2]["tool_calls"][0]["function"]["name"])

In [ ]:
# Review the reflection results
print("is_completed =", reflection_result["reflection_results"][0].is_completed)
print("advice =", reflection_result["reflection_results"][0].advice)